In [1]:
import pandas as pd
import json
import time


In [2]:
from openai import OpenAI


client = OpenAI()
#from error import APIConnectionError

In [3]:
df=pd.read_csv('../data/clean_data/final_data_with_IDs_new.csv')


In [4]:
df.head()

,id,content,number of sentences,number of words
0,0,Hello!. when you eat it alone?. Good idea to e...,20,296
1,1,its crazy how well im currently at reading my ...,20,382
2,2,"My minimum is 4 Personally* Well, nothing for ...",20,317
3,3,Use a spiritual board 👍 I will do it too.. Lov...,20,621
4,4,So it is not super surprising.. I was cold at ...,20,472


In [7]:
documents = df.to_dict(orient='records')
documents[0:2]

[{'id': 0,
  'Chunked_Content': "when you eat it alone?. Good idea to eat it once wuth honey and once alone My kids eat it with the meal.They aren't at home btw meals I will do it during week end Yesterday I had nausea after several hours passed from second meal.Today I had zero hunger until 20h+ passed after previous that meal Today I made only one typical meal enhanced with 4 egg yolks",
  'number of sentences': 2,
  'number of words': 69},
 {'id': 1,
  'Chunked_Content': 'So they ate clay?. Act Charcoal will do the job?. 4,5 h passed I feel neutral, zero hunger and body warmed itself to confort temperature.. Before meal I felt colder than normal.. Thx Ring',
  'number of sentences': 5,
  'number of words': 34}]

In [10]:
prompt_template = """
You emulate a user of our primal health adviser application.
Formulate 5 questions this user might ask based on chunked content of conversation.
Make the questions specific to that chunk.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

Chunked_Content: {Chunked_Content}


Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [11]:
prompt = prompt_template.format(**documents[0])

In [12]:
prompt

'You emulate a user of our primal health adviser application.\nFormulate 5 questions this user might ask based on chunked content of conversation.\nMake the questions specific to that chunk.\nThe record should contain the answer to the questions, and the questions should\nbe complete and not too short. Use as fewer words as possible from the record. \n\nThe record:\n\nChunked_Content: when you eat it alone?. Good idea to eat it once wuth honey and once alone My kids eat it with the meal.They aren\'t at home btw meals I will do it during week end Yesterday I had nausea after several hours passed from second meal.Today I had zero hunger until 20h+ passed after previous that meal Today I made only one typical meal enhanced with 4 egg yolks\n\n\nProvide the output in parsable JSON without using code blocks:\n\n{"questions": ["question1", "question2", ..., "question5"]}'

In [13]:
print(_)

You emulate a user of our primal health adviser application.
Formulate 5 questions this user might ask based on chunked content of conversation.
Make the questions specific to that chunk.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

Chunked_Content: when you eat it alone?. Good idea to eat it once wuth honey and once alone My kids eat it with the meal.They aren't at home btw meals I will do it during week end Yesterday I had nausea after several hours passed from second meal.Today I had zero hunger until 20h+ passed after previous that meal Today I made only one typical meal enhanced with 4 egg yolks


Provide the output in parsable JSON without using code blocks:

{"questions": ["question1", "question2", ..., "question5"]}


In [14]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [15]:
questions = llm(prompt)
questions

'{"questions": ["Is it beneficial to eat the food with honey or alone?","When do your kids typically eat their meals?","What meal preparation are you planning for the weekend?","What did you experience after your second meal yesterday?","How did your hunger levels change today compared to yesterday?"]}'

In [16]:
import json

In [17]:
json.loads(questions)

{'questions': ['Is it beneficial to eat the food with honey or alone?',
  'When do your kids typically eat their meals?',
  'What meal preparation are you planning for the weekend?',
  'What did you experience after your second meal yesterday?',
  'How did your hunger levels change today compared to yesterday?']}

In [18]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [19]:
from tqdm.auto import tqdm

In [20]:
results = {}


In [21]:
len(documents)

5650

In [20]:
# Generationg questions for 49k documents was taking too long

In [22]:
# get 300 random documents out of 49006 documents for evaluation
import random
number =300                           # set the number to select here.
random_docs = random.sample(documents,number)
len(random_docs)


300

## Parallelarization
it did not work

In [23]:
from concurrent.futures import ThreadPoolExecutor, as_completed
def process_document(doc):
    doc_id = doc['id']
    if doc_id in results:
        return None  # Skip this document if already processed
    try:
        questions_raw = generate_questions(doc)
        questions = json.loads(questions_raw)
        return (doc_id, questions['questions'])
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error processing doc_id {doc_id}: {e}")
        return None

# Track time
t0 = time.time()

# Create a ThreadPoolExecutor with a suitable number of workers
with ThreadPoolExecutor(max_workers=10) as executor:
    # Submit all tasks
    
    futures = {executor.submit(process_document, doc): doc for doc in tqdm(random_docs)}

    total_tasks = len(futures)
    # Collect results as they complete
    for future in tqdm(as_completed(futures)):
        result = future.result()
        if result is not None:
            doc_id, questions = result
            results[doc_id] = questions

# End time
t1 = time.time()
# Print time taken
print(f"Time taken: {(t1 - t0)/60} minutes")


  0%|          | 0/300 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Time taken: 0.8857945680618287 minutes


In [24]:
len(results)


300

In [25]:
# no need to use the following code

In [26]:
# import time
# t0=time.time()
# for doc in tqdm(random_docs[:1000]): 
#     doc_id = doc['id']
#     if doc_id in results:
#         continue
#     try:
#         questions_raw = generate_questions(doc)
#         questions = json.loads(questions_raw)  # If an error happens here, the next line will not run
#         results[doc_id] = questions['questions']  # This line will be skipped if there's an error in json.loads()
#     except (json.JSONDecodeError, KeyError) as e:
#         print(f"Error processing doc_id {doc_id}: {e}")
    
# t1=time.time()
# print((t0-t1)/60)

In [27]:
len(results)
type(results)

dict

In [28]:
final_results = []

for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))

In [31]:
final_results[0:10]

[(3392, 'Why does cutting hair hurt so much?'),
 (3392, 'How can egg whites help in hairstyling?'),
 (3392, 'What are some alternatives to egg whites for hair styling?'),
 (3392, 'Is it common to avoid washing meat?'),
 (3392, 'What are the potential risks of not washing meat?'),
 (2766, 'What benefits does rotten kidney have for kidney health?'),
 (2766, 'How does fresh kidney contribute to kidney function?'),
 (2766, 'What is the purpose of making high meat?'),
 (2766, 'Can you explain how rotten kidney helps dissolve kidney toxicity?'),
 (2766,
  'Is there a specific way to prepare high meat for it to be effective?')]

In [33]:
df = pd.DataFrame(final_results, columns=['id', 'question'])
df.head()

,id,question
0,3392,Why does cutting hair hurt so much?
1,3392,How can egg whites help in hairstyling?
2,3392,What are some alternatives to egg whites for h...
3,3392,Is it common to avoid washing meat?
4,3392,What are the potential risks of not washing meat?


In [34]:
df.to_csv('../data/clean_data/ground-truth-data.csv', index=False)

In [35]:
!head '../data/clean_data/ground-truth-data.csv'

id,question
3392,Why does cutting hair hurt so much?
3392,How can egg whites help in hairstyling?
3392,What are some alternatives to egg whites for hair styling?
3392,Is it common to avoid washing meat?
3392,What are the potential risks of not washing meat?
2766,What benefits does rotten kidney have for kidney health?
2766,How does fresh kidney contribute to kidney function?
2766,What is the purpose of making high meat?
2766,Can you explain how rotten kidney helps dissolve kidney toxicity?
